# <font color='white'>Modelagem Estatística e Interpretação de Resultados na Análise de Churn</font>

## 1. Definição do Problema de Negócio

In [ ]:
display(Markdown("""
## 1.1. Problema de Negócio

A **Connecta Telecom**, uma empresa fictícia de telecomunicações, está enfrentando uma taxa de cancelamento de serviços (churn) acima da média do setor. A perda de clientes não só impacta a receita recorrente, mas também gera custos elevados com a aquisição de novos clientes para substituir os que foram perdidos.

A diretoria precisa de respostas claras e baseadas em dados para a seguinte pergunta: **"Quais são os principais fatores que levam nossos clientes a cancelar o serviço?"**

- Queremos compreender a relação entre variáveis -> Modelagem Estatística
- Se quisermos prever o cancelamento (churn) -> Modelagem Preditiva

## 1.2. Objetivos do Projeto

Este projeto visa utilizar a modelagem estatística para analisar dados de clientes e atingir os seguintes objetivos:

1.  **Identificar os Fatores-Chave:** Determinar quais variáveis (como tipo de contrato, tempo de fidelidade, valor da fatura) têm um impacto estatisticamente significativo na probabilidade de um cliente cancelar o serviço.
2.  **Quantificar o Impacto:** Medir o quão forte é a influência de cada fator no risco de churn.
3.  **Gerar Recomendações:** Traduzir os resultados da análise estatística em recomendações de negócio acionáveis para a criação de estratégias de retenção de clientes.

O modelo escolhido para esta análise será a **Regressão Logística**, pois o nosso objetivo é entender a relação entre diversas variáveis e uma variável de resultado binária (Churn: Sim ou Não).
"""))

## 2. Importação das Bibliotecas

In [ ]:
# Instala o pacote watermark
!pip install -q -U watermark

In [ ]:
# Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

In [ ]:
# Configurações para melhor visualização
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
%reload_ext watermark
%watermark -a "Alex_AI_Solutions"

In [ ]:
%watermark --iversions

## 3. Extração dos Dados

In [ ]:
# Função para gerar dados fictícios mas coerentes para análise
def dsa_gera_dados_churn(num_clientes = 2000):
    
    """
    Gera um DataFrame de dados fictícios de clientes de uma empresa de telecomunicações.
    """

    # Para reprodutibilidade
    np.random.seed(42)  

    # Variáveis
    fidelidade_meses = np.random.randint(1, 73, size = num_clientes)
    tipo_contrato_opts = ['Mensal', 'Anual', 'Dois anos']
    contrato_probs = [0.6, 0.25, 0.15]
    tipo_contrato = np.random.choice(tipo_contrato_opts, size = num_clientes, p = contrato_probs)
    servico_internet_opts = ['Fibra Óptica', 'DSL', 'Não']
    internet_probs = [0.55, 0.35, 0.10]
    servico_internet = np.random.choice(servico_internet_opts, size = num_clientes, p = internet_probs)

    fatura_base = {
        'Mensal': np.random.normal(60, 20),
        'Anual': np.random.normal(70, 25),
        'Dois anos': np.random.normal(80, 25)
    }
    
    fatura_mensal = [fatura_base[c] + fidelidade_meses[i] * 0.2 + np.random.normal(0, 5) for i, c in enumerate(tipo_contrato)]
    fatura_mensal = np.clip(fatura_mensal, 20, 120)

    # Lógica para a probabilidade de Churn
    # Clientes com contrato mensal, baixa fidelidade e fatura alta têm maior chance de churn
    prob_churn_log = -2.5  # Intercepto base (tendência a não cancelar)
    prob_churn_log += -0.05 * fidelidade_meses  # Mais fidelidade, menor chance
    prob_churn_log += [3.0 if c == 'Mensal' else -1.5 if c == 'Anual' else -2.5 for c in tipo_contrato] # Contrato mensal aumenta muito a chance
    prob_churn_log += [0.8 if s == 'Fibra Óptica' else -0.5 for s in servico_internet] # Fibra tende a ter mais churn (talvez por preço)
    prob_churn_log += 0.03 * fatura_mensal # Fatura mais alta, mais chance
    
    # Converter log-odds para probabilidade usando a função sigmoide
    prob_churn = 1 / (1 + np.exp(-prob_churn_log))
    
    # Gerar o resultado de churn com base na probabilidade
    churn = np.random.binomial(1, prob_churn)

    df = pd.DataFrame({
        'ID_Cliente': range(1, num_clientes + 1),
        'Fidelidade_Meses': fidelidade_meses,
        'Tipo_Contrato': tipo_contrato,
        'Servico_Internet': servico_internet,
        'Fatura_Mensal': fatura_mensal,
        'Churn': churn
    })

    return df

In [ ]:
# Gera os dados
df_churn = dsa_gera_dados_churn()

In [ ]:
display(Markdown("### Amostra dos Dados Gerados"))
df_churn.head()

## 4. Análise Exploratória dos Dados (EDA)

In [ ]:
# Informações gerais do DataFrame
display(Markdown("### Informações Gerais do DataFrame"))
df_churn.info()

In [ ]:
# Resumo estatístico
display(Markdown("### Resumo Estatístico das Variáveis Numéricas"))
display(df_churn.describe())

In [ ]:
# Resumo estatístico
display(Markdown("### Resumo Estatístico das Variáveis Categóricas"))
display(df_churn.describe(include='object'))

### 4.1. Gráfico de Taxa de Churn Geral

In [ ]:
# Gráfico - Versão Estática

# Conta os valores da coluna 'Churn'
churn_counts = df_churn['Churn'].value_counts().rename(index = {1: 'Sim', 0: 'Não'})

# Define as cores equivalentes ao gráfico original
cores = ['#636EFA', '#EF553B']

# Cria o gráfico de pizza com duas casas decimais
plt.figure(figsize = (6,6))
plt.pie(
    churn_counts.values, 
    labels = churn_counts.index, 
    autopct = '%1.2f%%',   # <-- Duas casas decimais
    startangle = 140, 
    colors = cores,
    explode = [0.05 if label == 'Sim' else 0 for label in churn_counts.index]
)
plt.title('Taxa de Churn Geral', fontsize = 14)
plt.show()

In [ ]:
# Gráfico - Versão Interativa

# Calcula a contagem por categoria
churn_counts = df_churn['Churn'].value_counts() 

# Cria o gráfico
fig_pie = px.pie(values = churn_counts.values, 
                 names = churn_counts.index.map({1: 'Sim', 0: 'Não'}), 
                 title = 'Taxa de Churn Geral', 
                 color = churn_counts.index.map({1: 'Sim', 0: 'Não'}), 
                 color_discrete_map = {'Sim':'#EF553B', 'Não':'#636EFA'} ) 
fig_pie.show()

In [ ]:
# Calculamos a taxa de churn
churn_counts = df_churn['Churn'].value_counts()
numerador = churn_counts.get(1, churn_counts.get('Sim', churn_counts.get(True, 0)))
taxa = 100 * numerador / len(df_churn)

display(Markdown(f"""
A taxa de churn geral na nossa base de dados é de **{taxa:.2f}%**.
Este é o nosso principal indicador a ser analisado.
"""))

### 4.2. Gráfico de Taxa de Churn Por Tipo de Contrato

In [ ]:
# Gráfico - Versão Estática

plt.figure(figsize = (12,4))

# Cria o gráfico de barras agrupadas com Seaborn
sns.countplot(data = df_churn,
              x = 'Tipo_Contrato',
              hue = 'Churn',
              palette = {0: '#636EFA', 1: '#EF553B'})

plt.title('Taxa de Churn Por Tipo de Contrato', fontsize = 14)
plt.xlabel('\nTipo de Contrato')
plt.ylabel('Número de Clientes')
plt.legend(title = 'Churn (0=Não, 1=Sim)')
plt.xticks(rotation = 0)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico - Versão Interativa

# Histograma
dsa_fig_bar_contrato = px.histogram(df_churn, 
                                    x = 'Tipo_Contrato', 
                                    color = 'Churn', 
                                    barmode = 'group',
                                    title = 'Taxa de Churn por Tipo de Contrato',
                                    labels = {'Tipo_Contrato': 'Tipo de Contrato', 'Churn': 'Churn (0=Não, 1=Sim)'})
dsa_fig_bar_contrato.show()

In [ ]:
display(Markdown("""
**Insight:** A visualização é clara: clientes com **contrato mensal** têm uma propensão ao churn drasticamente maior em comparação com clientes de contratos anuais ou de dois anos.
"""))

### 4.3. Gráfico de Distribuição da Fidelidade (em Meses) Por Churn

In [ ]:
# Gráfico - Versão Interativa

# Análise por Fidelidade (Tenure)
dsa_fig_hist_fidelidade = px.histogram(df_churn, 
                                       x = 'Fidelidade_Meses', 
                                       color = 'Churn', 
                                       marginal = 'box',
                                       title = 'Distribuição da Fidelidade (em Meses) Por Churn',
                                       labels = {'Fidelidade_Meses': 'Meses de Fidelidade'})
dsa_fig_hist_fidelidade.show()

In [ ]:
display(Markdown("""
**Insight:** Clientes que cancelam o serviço (`Churn=1`) estão mais concentrados nos primeiros meses de contrato. A mediana de fidelidade para clientes que cancelam é muito menor do que para os que permanecem. Isso indica que o início da jornada do cliente é um período crítico para a retenção.
"""))

### 4.4. Gráfico de Distribuição da Fatura Mensal Por Churn

In [ ]:
# Gráfico - Versão Interativa

# Análise por Fatura Mensal
dsa_fig_hist_fatura = px.histogram(df_churn, 
                                   x = 'Fatura_Mensal', 
                                   color = 'Churn', 
                                   marginal = 'box',
                                   title = 'Distribuição da Fatura Mensal Por Churn',
                                   labels = {'Fatura_Mensal': 'Valor da Fatura Mensal'})
dsa_fig_hist_fatura.show()

In [ ]:
display(Markdown("""
**Insight:** Clientes com faturas mensais mais altas tendem a apresentar uma taxa de churn maior. A mediana do valor da fatura para clientes que cancelaram é notavelmente superior.
"""))

## 5. Preparação dos Dados Para a Modelagem Estatística

In [ ]:
# Dados originais
df_churn.head()

In [ ]:
# Categorias da variável
df_churn.Tipo_Contrato.value_counts()

In [ ]:
# Categorias da variável
df_churn.Servico_Internet.value_counts()

In [ ]:
# Preparação dos dados
# Converter variáveis categóricas em variáveis dummy (0 ou 1)
# Adicionado o parâmetro dtype=int para garantir que as novas colunas sejam numéricas
df_model = pd.get_dummies(df_churn, columns = ['Tipo_Contrato', 'Servico_Internet'], drop_first = True, dtype = int)

Variáveis dummy (ou variáveis indicadoras) são variáveis numéricas criadas para representar categorias em modelos estatísticos ou de Machine Learning. Elas transformam dados qualitativos (como “genero”, “estado civil”, “região”) em valores numéricos, geralmente 0 e 1, permitindo que algoritmos matemáticos consigam processá-los.

Assim, o modelo consegue interpretar diferenças entre grupos sem precisar lidar com texto. As variáveis dummy são uma forma de codificar categorias em números binários, facilitando análises estatísticas e predições. A técnica de criação de variáveis dummy é chamada de One-Hot-Encoding.

In [ ]:
# Dados originais
df_churn.head()

In [ ]:
# Dados processados
df_model.head()

In [ ]:
# Resumo dos dados
df_model.info()

In [ ]:
# Definir as variáveis
# Variável dependente (o que queremos prever)
y = df_model['Churn']

In [ ]:
# Variáveis independentes (as que usamos para prever)
# Excluímos a ID do cliente e a variável alvo original
X = df_model.drop(['ID_Cliente', 'Churn'], axis = 1)

In [ ]:
# Adicionar uma constante (intercepto) ao modelo, exigido pela biblioteca statsmodels
X = sm.add_constant(X)

In [ ]:
display(Markdown("### Amostra dos Dados (X) Preparados Para o Modelo"))
display(X.head())

In [ ]:
display(Markdown("### Amostra dos Dados (y) Preparados Para o Modelo"))
display(y.head())

## 6. Modelagem Estatística

In [ ]:
display(Markdown("Vamos construir um modelo para quantificar o impacto de cada variável na probabilidade de churn."))

In [ ]:
# Criamos o modelo
modelo = sm.Logit(y, X)

In [ ]:
type(modelo)

In [ ]:
# Treinamento do modelo
modelo_treinado = modelo.fit()

In [ ]:
# Exibir o resumo completo do modelo
display(Markdown("### Resultados do Modelo de Regressão Logística"))
modelo_treinado.summary()

## 7. Interpretação dos Resultados

In [ ]:
display(Markdown("""
O resumo acima contém informações ricas sobre o nosso modelo. Vamos focar nos pontos mais importantes para o negócio:

1.  **Coeficientes (`coef`):** Mostram a direção do impacto. Um coeficiente positivo aumenta a chance de churn, enquanto um negativo a diminui.
2.  **Valor-p (`P>|z|`):** Indica a significância estatística de cada variável. Um valor-p baixo (geralmente < 0.05) significa que o efeito da variável é real e não apenas uma coincidência na amostra.
3.  **Pseudo R-squ.:** Similar ao R² na regressão linear, indica a proporção da "variância" da variável dependente que é explicada pelo modelo. Um valor de 0.645 (ou 65%) é considerado um bom ajuste para este tipo de modelo.

Para facilitar a interpretação, vamos calcular a **Razão de Chances (Odds Ratio)**, que nos diz como as chances de churn mudam com o aumento de uma unidade na variável independente.
"""))

A Razão de Chances (ou Odds Ratio) é uma medida usada para comparar a probabilidade de um evento ocorrer entre dois grupos. Ela é muito comum em estatística e modelos como a regressão logística.

De forma simples, a “chance” (odds) de um evento é a razão entre a probabilidade de ele acontecer e a probabilidade de não acontecer. Por exemplo, se a probabilidade de um cliente comprar um produto é 0,8 (80%), então a chance é 0,8 / 0,2 = 4. Isso quer dizer que a chance de compra é 4 vezes maior do que a de não compra.

In [ ]:
# Calcular e exibir as razões de chance (Odds Ratios)
params = modelo_treinado.params
conf = modelo_treinado.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
conf = np.exp(conf)
display(Markdown("### Tabela de Razão de Chances (Odds Ratio)"))
display(conf)

## Interpretação da Razão de Chances Para o Negócio:

**1. const (intercepto)**

O valor 0.0054 representa a chance base de churn quando todas as outras variáveis são zero. Ele por si só não tem interpretação prática relevante e serve como ponto de partida do modelo.

**2. Fidelidade_Meses (0.9459)**

O odds ratio é menor que 1, o que indica um efeito protetor: quanto mais meses de fidelidade, menor a chance de churn. Cada mês adicional reduz em cerca de (1 - 0.9459) × 100 ≈ 5,4% a chance de o cliente cancelar.

**3. Fatura_Mensal (1.0321)**

Um valor ligeiramente acima de 1 mostra que quanto maior a fatura mensal, maior a chance de churn. A cada aumento unitário (por exemplo, R$ 1), a chance cresce em torno de 3,2%. Isso pode indicar insatisfação com preços mais altos.

**4. Tipo_Contrato_Dois anos (0.2797)**

O valor é menor que 1, o que significa que clientes com contrato de dois anos têm menor probabilidade de cancelar comparados ao grupo de referência (por exemplo “Contrato de um ano”). Isso sugere que contratos longos retêm melhor os clientes.

**5. Tipo_Contrato_Mensal (195.1849)**

Um valor muito alto mostra que clientes com contrato mensal têm probabilidade muito maior de cancelar, cerca de 195 vezes maior do que o grupo de referência. Isso é típico: contratos mensais são mais flexíveis e os clientes saem mais facilmente.

**6. Servico_Internet_Fibra Óptica (4.2844)**

Clientes com fibra óptica têm 4,28 vezes mais chance de churn que os que usam outro tipo de internet (como DSL, que seria o grupo base). Pode indicar que esse grupo é mais exigente ou sensível a falhas de serviço.

**7. Servico_Internet_Não (1.2331)**

Clientes sem serviço de internet têm uma chance 23% maior de churn em relação ao grupo base, mas o intervalo de confiança (0.73 a 2.06) inclui 1, o que sugere baixa significância estatística, ou seja, o efeito pode não ser real.

Em resumo:

- Fidelidade e contrato de longo prazo reduzem churn.
- Fatura alta, contrato mensal e fibra óptica aumentam churn.
- O odds ratio mostra o tamanho e a direção do efeito de cada variável sobre a probabilidade de cancelamento.

## 8. Relatório Final - Conclusões e Recomendações Estratégicas

In [ ]:
display(Markdown("""
# Relatório Final: Conclusões e Recomendações Estratégicas

## Conclusões

A análise estatística revelou, com alto grau de confiança, os principais impulsionadores do churn de clientes na **Connecta Telecom**. Os resultados do modelo de regressão logística confirmam que o cancelamento está fortemente associado a alguns fatores comportamentais e contratuais específicos. O perfil de cliente com **alto risco de churn** é: **cliente recente, com contrato mensal, fatura elevada e serviço de Fibra Óptica**.

Os fatores mais significativos, em ordem de impacto, são:
1. **Tipo de Contrato:** Clientes com contrato mensal apresentam um risco de cancelamento aproximadamente **195 vezes maior** em relação aos de contratos mais longos. Contratos de dois anos são o principal fator de proteção.
2. **Tempo de Fidelidade:** Cada mês adicional de permanência reduz a chance de churn em cerca de **5%**, mostrando que os primeiros meses são cruciais para a retenção.
3. **Valor da Fatura Mensal:** O aumento no valor da fatura mensal eleva o risco de churn em aproximadamente **3% por unidade monetária**, sugerindo sensibilidade a preço.
4. **Tipo de Serviço de Internet:** O serviço de **Fibra Óptica** aumenta em cerca de **4 vezes** a chance de cancelamento em comparação com o DSL, possivelmente devido a expectativas não atendidas ou à percepção de custo elevado.

## Recomendações Acionáveis

Com base nessas conclusões, propõem-se as seguintes ações estratégicas para o time de retenção:

| Recomendação | Justificativa (Baseada nos Dados) | Ação Proposta |
| :--- | :--- | :--- |
| **1. Incentivar a Migração para Contratos de Longo Prazo** | O contrato mensal está associado a um risco **extremamente alto de churn (≈195x)**. | Criar uma campanha promocional para converter clientes mensais em anuais ou bienais, oferecendo incentivos como descontos, bônus de fidelidade ou upgrades temporários de velocidade. |
| **2. Criar um Programa de Fidelização para Novos Clientes** | Cada mês adicional reduz a chance de churn em **5%**. | Implementar um programa de acompanhamento intensivo nos seis primeiros meses, com comunicações personalizadas, suporte proativo e benefícios progressivos para reforçar a satisfação inicial. |
| **3. Revisar a Estratégia de Preços e Valor Percebido** | Faturas mais altas aumentam a chance de churn em **3%** por unidade. | Segmentar clientes com faturas elevadas e realizar uma análise de elasticidade de preço. Ajustar pacotes para melhorar o valor percebido (mais benefícios pelo mesmo preço). |
| **4. Investigar o Churn no Segmento de Fibra Óptica** | Clientes com Fibra têm **4x mais chances de cancelar**. | Conduzir uma pesquisa de satisfação e uma análise técnica detalhada sobre a qualidade do serviço e suporte. Identificar causas de insatisfação e agir sobre problemas de performance ou atendimento. |

A adoção dessas recomendações, baseadas em evidências estatísticas sólidas e profissionais, permitirá à **Connecta Telecom** reduzir substancialmente o churn, aumentar a fidelização e fortalecer sua base de receita recorrente.
"""))